In [1]:
!pip install -qU "semantic-router[pinecone]>=0.1.5"

# Syncing Routes with Pinecone Index

When using the `PineconeIndex`, our `RouteLayer` is stored in two places:

* We keep route layer metadata locally.
* Vectors alongside a backup of our metadata is stored remotely in Pinecone.

By storing some data locally and some remotely we achieve improved persistence and the ability to recover our local state if lost. However, it does come with challenges around keep our local and remote instances synchronized. Fortunately, we have [several synchronization options](https://docs.aurelio.ai/semantic-router/route_layer/sync.html). In this example, we'll see how to use these options to keep our local and remote Pinecone instances synchronized.

In [2]:
from semantic_router import Route

# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat]

/Users/jamesbriggs/Library/Caches/pypoetry/virtualenvs/semantic-router-C1zr4a78-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

# get at platform.openai.com
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API key: "
)

encoder = OpenAIEncoder(name="text-embedding-3-small")

For our `PineconeIndex` we do the exact same thing, ie we initialize as usual:

In [4]:
import os
from semantic_router.index.pinecone import PineconeIndex

# get at app.pinecone.io
os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)

pc_index = PineconeIndex(
    dimensions=1536,
    init_async_index=True,  # enables asynchronous methods, it's optional
)
pc_index.index = pc_index._init_index(force_create=True)

## SemanticRouter

The `SemanticRouter` class supports both sync and async operations by default, so we initialize as usual:

In [5]:
from semantic_router.routers import SemanticRouter
import time

rl = SemanticRouter(encoder=encoder, routes=routes, index=pc_index, auto_sync="local")
# due to pinecone indexing latency we wait 3 seconds
time.sleep(3)

2024-11-23 23:46:42 WARNING semantic_router.utils.logger TEMP | add:
chitchat: how are things going?
chitchat: how's the weather today?
chitchat: let's go to the chippy
chitchat: lovely weather today
chitchat: the weather is horrendous
2024-11-23 23:46:50 WARNING semantic_router.utils.logger TEMP | add:
chitchat: how are things going?
chitchat: how's the weather today?
chitchat: let's go to the chippy
chitchat: lovely weather today
chitchat: the weather is horrendous


Let's see if our local and remote instances are synchronized...

In [6]:
rl.is_synced()

True

It looks like everything is synced! Let's try deleting our local route layer, initializing it with just the politics route, and checking again.

In [7]:
del rl

rl = SemanticRouter(encoder=encoder, routes=[politics], index=pc_index)
time.sleep(3)

Let's try `rl.is_synced()` again:

In [8]:
rl.is_synced()

False

We can use the `get_utterance_diff` method to see exactly _why_ our local and remote are not synced

In [9]:
rl.get_utterance_diff()

['+ chitchat: how are things going?',
 "+ chitchat: how's the weather today?",
 "+ chitchat: let's go to the chippy",
 '+ chitchat: lovely weather today',
 '+ chitchat: the weather is horrendous',
 "  politics: don't you just hate the president",
 "  politics: don't you just love the president",
 "  politics: isn't politics the best thing ever",
 '  politics: they will save the country!',
 "  politics: they're going to destroy this country!",
 "  politics: why don't you tell me about your political opinions"]

## Handling Synchronization

We may want to handle the resynchronization ourselves and to do that we ideally want a more structured version of the utterance diff returned above. To create that we first need to get a list of utterance objects from our remote and local instances:

In [10]:
remote_utterances = rl.index.get_utterances()
remote_utterances

[Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='chitchat', utterance='the weather is horrendous', function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='chitchat', utterance='lovely weather today', function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="don't you just hate the president", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="don't you just love the president", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="they're going to destroy this country!", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utt

In [11]:
local_utterances = rl.to_config().to_utterances()
local_utterances

[Utterance(route='politics', utterance="isn't politics the best thing ever", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="why don't you tell me about your political opinions", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="don't you just love the president", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="don't you just hate the president", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="they're going to destroy this country!", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance='they will save the country!', function_schemas=None, metadata={}, diff_tag=' ')]

We can add the `diff_tag` attribute to each of these utterances by loading both lists into a `UtteranceDiff` object:

In [12]:
from semantic_router.schema import UtteranceDiff

diff = UtteranceDiff.from_utterances(
    local_utterances=local_utterances, remote_utterances=remote_utterances
)

`UtteranceDiff` objects include all diff information inside the `diff` attribute (which is a list of `Utterance` objects):

In [13]:
diff.diff

[Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='chitchat', utterance='lovely weather today', function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='chitchat', utterance='the weather is horrendous', function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='politics', utterance="don't you just hate the president", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="don't you just love the president", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="isn't politics the best thing ever", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utteran

Each of our `Utterance` objects now contains a populate `diff_tag` attribute. Where:

* `diff_tag='+'` means the utterance exists in the remote instance *only*

* `diff_tag='-'` means the utterance exists in the local instance *only*

* `diff_tag=' '` means the utterance exists in both remote and local instances

So, to collect utterances missing from our local instance we can run:

In [14]:
diff.get_tag("+")

[Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='chitchat', utterance='lovely weather today', function_schemas=None, metadata={}, diff_tag='+'),
 Utterance(route='chitchat', utterance='the weather is horrendous', function_schemas=None, metadata={}, diff_tag='+')]

To collect utterances missing from our remote instance we can run:

In [15]:
diff.get_tag("-")

[]

And, if needed, we can get all utterances that exist in both with:

In [16]:
diff.get_tag(" ")

[Utterance(route='politics', utterance="don't you just hate the president", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="don't you just love the president", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="isn't politics the best thing ever", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance='they will save the country!', function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="they're going to destroy this country!", function_schemas=None, metadata={}, diff_tag=' '),
 Utterance(route='politics', utterance="why don't you tell me about your political opinions", function_schemas=None, metadata={}, diff_tag=' ')]

## Synchronization

There are six synchronization methods that we can use, those are:

* `error`: Raise an error if local and remote are not synchronized.

* `remote`: Take remote as the source of truth and update local to align.

* `local`: Take local as the source of truth and update remote to align.

* `merge-force-remote`: Merge both local and remote keeping local as the priority. Remote utterances are only merged into local *if* a matching route for the utterance is found in local, all other route-utterances are dropped. Where a route exists in both local and remote, but each contains different `function_schema` or `metadata` information, the local version takes priority and local `function_schemas` and `metadata` is propogated to all remote utterances belonging to the given route.

* `merge-force-local`: Merge both local and remote keeping remote as the priority. Local utterances are only merged into remote *if* a matching route for the utterance is found in the remote, all other route-utterances are dropped. Where a route exists in both local and remote, but each contains different `function_schema` or `metadata` information, the remote version takes priotity and remote `function_schemas` and `metadata` are propogated to all local routes.

* `merge`: Merge both local and remote, merging also local and remote utterances when a route with same route name is present both locally and remotely. If a route exists in both local and remote but contains different `function_schemas` or `metadata` information, the local version takes priority and local `function_schemas` and `metadata` are propogated to all remote routes.

We can get the synchronization strategy for each of these (with the exception of `error`) using the `diff.get_sync_strategy` method.

In [17]:
diff.get_sync_strategy("local")

{'remote': {'upsert': [],
  'delete': [Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='lovely weather today', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='the weather is horrendous', function_schemas=None, metadata={}, diff_tag='+')]},
 'local': {'upsert': [], 'delete': []}}

In [18]:
diff.get_sync_strategy("remote")

{'remote': {'upsert': [], 'delete': []},
 'local': {'upsert': [Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='lovely weather today', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='the weather is horrendous', function_schemas=None, metadata={}, diff_tag='+')],
  'delete': []}}

In [19]:
diff.get_sync_strategy("merge-force-remote")

{'remote': {'upsert': [], 'delete': []},
 'local': {'upsert': [Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='lovely weather today', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='the weather is horrendous', function_schemas=None, metadata={}, diff_tag='+')],
  'delete': []}}

In [20]:
diff.get_sync_strategy("merge-force-local")

2024-11-23 23:47:11 INFO semantic_router.utils.logger local_only_mapper: {}


{'remote': {'upsert': [],
  'delete': [Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='lovely weather today', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='the weather is horrendous', function_schemas=None, metadata={}, diff_tag='+')]},
 'local': {'upsert': [], 'delete': []}}

In [21]:
diff.get_sync_strategy("merge")

{'remote': {'upsert': [Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='lovely weather today', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance='the weather is horrendous', function_schemas=None, metadata={}, diff_tag='+')],
  'delete': []},
 'local': {'upsert': [Utterance(route='chitchat', utterance='how are things going?', function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="how's the weather today?", function_schemas=None, metadata={}, diff_tag='+'),
   Utterance(route='chitchat', utterance="let's go to the chippy", function_schemas=None, metadata={}, diff_tag='+

Each of these sync strategies can be fed to our route layer via the `rl._execute_sync_strategy` method:

In [22]:
strategy = diff.get_sync_strategy("local")
rl._execute_sync_strategy(strategy=strategy)

2024-11-23 23:47:15 WARNING semantic_router.utils.logger TEMP | _remove_and_sync:
chitchat: ['how are things going?', "how's the weather today?", "let's go to the chippy", 'lovely weather today', 'the weather is horrendous']


In [23]:
time.sleep(3)
rl.is_synced()

True

We can check our diff method to see what the `local` sync did:

In [24]:
rl.get_utterance_diff()

["  politics: don't you just hate the president",
 "  politics: don't you just love the president",
 "  politics: isn't politics the best thing ever",
 '  politics: they will save the country!',
 "  politics: they're going to destroy this country!",
 "  politics: why don't you tell me about your political opinions"]

As expected, it took all local utterances and applied them to the remote instance, removing all utterances that were only present in the remote instance.

We can simplify this process significantly by running the `rl.sync` method with our chosen `sync_mode`:

In [25]:
rl.sync(sync_mode="local")

2024-11-23 23:47:23 WARNING semantic_router.utils.logger Local and remote route layers are already synchronized.


["  politics: don't you just hate the president",
 "  politics: don't you just love the president",
 "  politics: isn't politics the best thing ever",
 '  politics: they will save the country!',
 "  politics: they're going to destroy this country!",
 "  politics: why don't you tell me about your political opinions"]

---